# Item-based Collaborative Filtering

Core idea
“If two movies get similar rating patterns from many users, then someone who liked one of those movies will probably like the other as well.”

How it works
  1. For every movie the target user has rated, find similar movies (e.g., by cosine similarity of rating vectors).
  2. Score those similar movies—weight by how much the user liked the original movie and by the similarity strength.
  3. Rank the unseen movies by the aggregated scores.
  4. Recommend the top-ranked ones to the user.

Example
Many users who liked Inception also liked Interstellar and The Matrix.
Alice rated Inception and The Matrix highly but hasn’t watched Interstellar.
Because both of Alice’s liked movies point to Interstellar as a close neighbour, the system recommends Interstellar to Alice.

In [2]:
# Load datasets
import pandas as pd
movies = pd.read_csv("../data/csv/movies.csv")
ratings = pd.read_csv("../data/csv/ratings.csv")

In [3]:
# Merge ratings with movie titles
movies_ratings = ratings.merge(movies[['movieId', 'title', 'genres']], on='movieId', how='left')

print(movies_ratings.shape)
movies_ratings.head()

(25000095, 6)


,userId,movieId,rating,timestamp,title,genres
0,1,296,5.0,1147880044,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
1,1,306,3.5,1147868817,Three Colors: Red (Trois couleurs: Rouge) (1994),Drama
2,1,307,5.0,1147868828,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
3,1,665,5.0,1147878820,Underground (1995),Comedy|Drama|War
4,1,899,3.5,1147868510,Singin' in the Rain (1952),Comedy|Musical|Romance


## Filter to “Active” Users and “Popular” Movies

We do this, because the full dataset is too computationally expensive for personal laptops.

In [4]:
# Keep users with at least 500 ratings
user_counts = movies_ratings['userId'].value_counts()
active_users = user_counts[user_counts >= 500].index

# Keep movies with at least 1000 ratings
movie_counts = movies_ratings['movieId'].value_counts()
popular_movies = movie_counts[movie_counts >= 1000].index

# Filter the DataFrame
movies_ratings_filtered = movies_ratings[
    movies_ratings['userId'].isin(active_users) &
    movies_ratings['movieId'].isin(popular_movies)
]

print(movies_ratings_filtered.shape)
movies_ratings_filtered.head()

(7127698, 6)


,userId,movieId,rating,timestamp,title,genres
254,3,1,4.0,1439472215,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
255,3,29,4.5,1484754967,"City of Lost Children, The (Cité des enfants p...",Adventure|Drama|Fantasy|Mystery|Sci-Fi
256,3,32,4.5,1439474635,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
257,3,50,5.0,1439474391,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
258,3,111,4.0,1484753849,Taxi Driver (1976),Crime|Drama|Thriller


## Lenskit implementation

In [5]:
from lenskit.data import from_interactions_df

# convert df to a Dataset (new in LensKit 2025.2.0)
# DOCS: https://lkpy.lenskit.org/stable/guide/data/
lk_dataset = from_interactions_df(movies_ratings_filtered, 
                                   user_col='userId', 
                                   item_col='movieId', 
                                   rating_col='rating', 
                                   timestamp_col='timestamp')
lk_dataset
pd_lk_dataset = lk_dataset.interaction_matrix(format='pandas')
pd_lk_dataset

,user_num,item_num,rating,timestamp,title,genres
0,0,0,4.0,1439472215,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,0,28,4.5,1484754967,"City of Lost Children, The (Cité des enfants p...",Adventure|Drama|Fantasy|Mystery|Sci-Fi
2,0,31,4.5,1439474635,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
3,0,44,5.0,1439474391,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
4,0,84,4.0,1484753849,Taxi Driver (1976),Crime|Drama|Thriller
...,...,...,...,...,...,...
7127693,9712,2131,3.0,1000946228,American Pie 2 (2001),Comedy
7127694,9712,2138,4.0,1000948027,Jay and Silent Bob Strike Back (2001),Adventure|Comedy
7127695,9712,2156,5.0,1001884147,Dirty Harry (1971),Action|Crime|Thriller
7127696,9712,2157,5.0,1001886067,Fiddler on the Roof (1971),Drama|Musical


In [6]:
# we also can get some statistics from the Dataset object 
lk_dataset.item_stats()
# lk_dataset.user_stats()

/home/toporek3112/miniconda3/envs/ml-3.13/lib/python3.13/site-packages/lenskit/data/relationships.py:588: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'NaT' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  stats.loc[stats["count"] == 0, "first_time"] = pd.NaT
/home/toporek3112/miniconda3/envs/ml-3.13/lib/python3.13/site-packages/lenskit/data/relationships.py:590: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'NaT' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  stats.loc[stats["count"] == 0, "last_time"] = pd.NaT


,record_count,user_count,rating_count,mean_rating,count,first_time,last_time
item_id,,,,,,,
1,8114,8114,8114,3.828691,8114,828047324,1574112354
2,5881,5881,5881,3.059344,5881,828047324,1574009709
3,2289,2289,2289,2.850371,2289,833483749,1573255519
4,631,631,631,2.546751,631,833481887,1574213055
5,2244,2244,2244,2.707665,2244,833848976,1573033018
...,...,...,...,...,...,...,...
192389,566,566,566,3.130742,566,1538600291,1574112355
192803,802,802,802,3.614713,802,1540563118,1574314310
194448,653,653,653,3.822358,653,1542520104,1574171260


### Basic recommendation

In [ ]:
# split into test and train sets
from lenskit.splitting import sample_users, SampleFrac

# DOCS: https://lkpy.lenskit.org/stable/api/lenskit.splitting.sample_users
split = sample_users(lk_dataset, rng=42, method=SampleFrac(0.2), size=1000) 
"""
Splits the dataset based on users interactions. 
The `method=SampleFrac(0.2),` means that for each user, 20% of their interactions will be used for testing, and the rest will be used for training. 
"""

print(f"Train size: {split.train.interaction_count}, Test size: {len(split.test)}")

Train size: 6982451, Test size: 1000


In [8]:
# Build recommendation pipeline and train
from lenskit.knn import ItemKNNScorer
from lenskit.pipeline import RecPipelineBuilder
from lenskit.basic import UnratedTrainingItemsCandidateSelector
from lenskit import recommend

# 1. Initialize the pipeline builder
# DOCS: https://lkpy.lenskit.org/stable/api/pipeline.html#
builder = RecPipelineBuilder()

# 2. Add the item-item CF scoring model 
# DOCS: https://lkpy.lenskit.org/stable/api/lenskit.knn.item.html#lenskit.knn.item.ItemKNNScorer
scorer = ItemKNNScorer(k=20) 
builder.scorer(scorer)
# Training described: https://github.com/lenskit/lkpy/blob/16e5fc7dc8056dc3c55d2349c7bfa21565f4fe40/src/lenskit/knn/item.py#L131

# 3. Set the candidate selector to filter out items the user has rated
builder.candidate_selector(UnratedTrainingItemsCandidateSelector())
# DOCS: https://lkpy.lenskit.org/stable/api/lenskit.basic.html#lenskit.basic.UnratedTrainingItemsCandidateSelector

# 4. Set the ranker to produce Top-N recommendations (e.g., Top-10)
builder.ranker(n=10) 

# 5. Build the pipeline
pipe = builder.build("Simple ItemKNN Pipeline")

# 6. Train
pipe.train(split.train)

/home/toporek3112/miniconda3/envs/ml-3.13/lib/python3.13/site-packages/lenskit/data/relationships.py:378: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at /opt/conda/conda-bld/libtorch_1745854776362/work/aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  return torch.sparse_csr_tensor(


In [9]:
# batch recommend to users in test set
from lenskit.batch import recommend as batch_recommend

# DOCS: https://lkpy.lenskit.org/stable/guide/batch
rec = batch_recommend(pipe, list(split.test.keys()), n=10) 

In [10]:
# define functions to measure performance
from lenskit.metrics import RunAnalysis, Precision, Recall, Hit, NDCG
from sklearn.metrics import mean_squared_error
from lenskit.data import ItemListCollection, UserIDKey

analysis = RunAnalysis()
analysis.add_metric(Precision())
analysis.add_metric(Recall())
analysis.add_metric(NDCG())
analysis.add_metric(Hit())

def measure_performance(test: ItemListCollection, rec: ItemListCollection[UserIDKey]):
  df_rec = rec.to_df()
  df_test = test.to_df()

  # keep only the columns we need and join on user & item
  hits = (
    df_test[['user_id', 'item_id', 'rating']]
      .merge(df_rec[['user_id', 'item_id', 'score']],
            on=['user_id', 'item_id'],
            how='inner')          # drop pairs without predictions
  )

  mse  = mean_squared_error(hits['rating'], hits['score'])
  rmse = mse ** 0.5 

  # Measure the recommendations against the test data
  results = analysis.measure(rec, test)
  metrics = results.list_metrics().mean()             # Series: metric → mean value

  # build single-row DataFrame and append MSE / RMSE
  df = metrics.to_frame().T                        # rows → columns
  df['MSE']  = mse
  df['RMSE'] = rmse
  return df

measure_performance(split.test, rec)


,Precision,Recall,NDCG,Hit,MSE,RMSE
0,0.2622,0.017959,0.048221,0.817,0.332913,0.576986


In [ ]:
# test recommendations for a specific user
user_id = lk_dataset.users.index[3]
rec = recommend(pipe, user_id, n=10)
df_rec = rec.to_df()

output_columns = ['movieId', 'title', 'genres']

print("Recommendations for user", user_id)
user_rec = df_rec.merge(
  movies[output_columns],   # just the needed cols
  left_on='item_id',
  right_on='movieId',
  how='left'
)[output_columns]

# Movies the user has already seen
seen = movies_ratings[movies_ratings['userId'] == user_id].sort_values('rating', ascending=False)[output_columns]

# Which recommendations accidentally overlap (should be empty!)
rec_seen = user_rec[user_rec['movieId'].isin(seen)]

print('Already seen recommendations (should be empty):\n', rec_seen)
assert rec_seen.empty, 'Candidate selector failed - user got already-seen movies'

user_rec

Recommendations for user 80
Already seen recommendations (should be empty):
 Empty DataFrame
Columns: [movieId, title, genres]
Index: []


,movieId,title,genres
0,170705,Band of Brothers (2001),Action|Drama|War
1,48516,"Departed, The (2006)",Crime|Drama|Thriller
2,6016,City of God (Cidade de Deus) (2002),Action|Adventure|Crime|Drama|Thriller
3,5959,Narc (2002),Crime|Drama|Thriller
4,7438,Kill Bill: Vol. 2 (2004),Action|Drama|Thriller
5,4226,Memento (2000),Mystery|Thriller
6,68157,Inglourious Basterds (2009),Action|Drama|War
7,2542,"Lock, Stock & Two Smoking Barrels (1998)",Comedy|Crime|Thriller
8,55765,American Gangster (2007),Crime|Drama|Thriller
9,166024,Whiplash (2013),(no genres listed)


In [12]:
seen

,movieId,title,genres
10723,6,Heat (1995),Action|Crime|Thriller
10724,10,GoldenEye (1995),Action|Adventure|Thriller
10725,16,Casino (1995),Crime|Drama
11081,2976,Bringing Out the Dead (1999),Drama
11194,4034,Traffic (2000),Crime|Drama|Thriller
...,...,...,...
11218,4266,"Forsaken, The (2001)",Horror
11235,4356,Gentlemen Prefer Blondes (1953),Comedy|Musical|Romance
11236,4366,Atlantis: The Lost Empire (2001),Adventure|Animation|Children|Fantasy
10722,2,Jumanji (1995),Adventure|Children|Fantasy


### Cross Validation

In [14]:
# perform a crossfold-validation 
from collections import defaultdict
from lenskit.data import MutableItemListCollection, UserIDKey
from lenskit.splitting import crossfold_users

# why only tuning min_sim hyperparameter: https://lkpy.lenskit.org/stable/api/lenskit.knn.item.html#lenskit.knn.item.ItemKNNScorer.train
param_grid = [1e-6, 0.005, 0.01, 0.05, 0.07, 0.1, 0.5] # 1e-6 is default
results = defaultdict(list) 

# DOCS: https://lkpy.lenskit.org/stable/api/lenskit.splitting.crossfold_users.html#lenskit.splitting.crossfold_users
folds = list(crossfold_users(lk_dataset, partitions=5, method=SampleFrac(0.2), rng=42))

for p in param_grid:
  all_test = MutableItemListCollection(UserIDKey)
  all_rec = MutableItemListCollection(UserIDKey)
  print(f'\n=== min_sim = {p} ===')

  # Build fresh pipeline for this fold
  builder = RecPipelineBuilder()
  builder.candidate_selector(UnratedTrainingItemsCandidateSelector())
  builder.ranker(n=10) 
  scorer = ItemKNNScorer(min_sim=p) 
  builder.scorer(scorer)
  pipe = builder.build(f"CV ItemKNN Pipeline {p}")

  for f, split in enumerate(folds):
      print(f"=== fold {f} ===")
      print(f"=== Train size: {split.train.interaction_count}, Test size: {len(split.test)} ===")

      algo = pipe.clone()
      algo.train(split.train)

      # Generate top-10 recommendations for each user in the test set of this fold
      user_ids = [k.user_id for k in split.test.keys()]
      print(f"=== Generating recommendations for {len(user_ids)} users ===")
      rec = batch_recommend(algo, user_ids, n=10)

      # results[k].append({'fold': f, 'test': split.test, 'rec': rec})
      all_test.add_from(split.test)
      all_rec.add_from(rec)
      print(f"=== recommendations: {len(rec)} ===")

  # Calculate recommended movies by user
  df_all_rec = all_rec.to_df()
  avg_rec_by_user = len(df_all_rec) / len(user_ids)

  results[p].append({
      'test': all_test,
      'rec': all_rec,
      'avg_rec': avg_rec_by_user
  })



=== min_sim = 1e-06 ===
=== fold 0 ===
=== Train size: 6838777, Test size: 1943 ===
2025-06-03 18:34:56 [warning  ] loaded pipeline does not match hash


/home/toporek3112/miniconda3/envs/ml-3.13/lib/python3.13/site-packages/lenskit/pipeline/_impl.py:233: PipelineWarning: loaded pipeline config does not match hash
  builder = PipelineBuilder.from_config(config)


=== Generating recommendations for 1943 users ===
=== recommendations: 1943 ===
=== fold 1 ===
=== Train size: 6842660, Test size: 1943 ===
2025-06-03 18:36:08 [warning  ] loaded pipeline does not match hash
=== Generating recommendations for 1943 users ===
=== recommendations: 1943 ===
=== fold 2 ===
=== Train size: 6844516, Test size: 1943 ===
2025-06-03 18:37:18 [warning  ] loaded pipeline does not match hash
=== Generating recommendations for 1943 users ===
=== recommendations: 1943 ===
=== fold 3 ===
=== Train size: 6843624, Test size: 1942 ===
2025-06-03 18:38:27 [warning  ] loaded pipeline does not match hash
=== Generating recommendations for 1942 users ===
=== recommendations: 1942 ===
=== fold 4 ===
=== Train size: 6843397, Test size: 1942 ===
2025-06-03 18:39:38 [warning  ] loaded pipeline does not match hash
=== Generating recommendations for 1942 users ===
=== recommendations: 1942 ===

=== min_sim = 0.005 ===
=== fold 0 ===
=== Train size: 6838777, Test size: 1943 ===
202

/home/toporek3112/miniconda3/envs/ml-3.13/lib/python3.13/site-packages/lenskit/pipeline/_impl.py:233: PipelineWarning: loaded pipeline config does not match hash
  builder = PipelineBuilder.from_config(config)


=== Generating recommendations for 1943 users ===
=== recommendations: 1943 ===
=== fold 1 ===
=== Train size: 6842660, Test size: 1943 ===
2025-06-03 18:42:01 [warning  ] loaded pipeline does not match hash
=== Generating recommendations for 1943 users ===
=== recommendations: 1943 ===
=== fold 2 ===
=== Train size: 6844516, Test size: 1943 ===
2025-06-03 18:43:19 [warning  ] loaded pipeline does not match hash
=== Generating recommendations for 1943 users ===
=== recommendations: 1943 ===
=== fold 3 ===
=== Train size: 6843624, Test size: 1942 ===
2025-06-03 18:44:32 [warning  ] loaded pipeline does not match hash
=== Generating recommendations for 1942 users ===
=== recommendations: 1942 ===
=== fold 4 ===
=== Train size: 6843397, Test size: 1942 ===
2025-06-03 18:45:41 [warning  ] loaded pipeline does not match hash
=== Generating recommendations for 1942 users ===
=== recommendations: 1942 ===

=== min_sim = 0.01 ===
=== fold 0 ===
=== Train size: 6838777, Test size: 1943 ===
2025

...

=== Generating recommendations for 1943 users ===
=== recommendations: 1943 ===
=== fold 1 ===
=== Train size: 6842660, Test size: 1943 ===
2025-06-03 18:58:12 [warning  ] loaded pipeline does not match hash
=== Generating recommendations for 1943 users ===
=== recommendations: 1943 ===
=== fold 2 ===
=== Train size: 6844516, Test size: 1943 ===
2025-06-03 18:59:02 [warning  ] loaded pipeline does not match hash
=== Generating recommendations for 1943 users ===
=== recommendations: 1943 ===
=== fold 3 ===
=== Train size: 6843624, Test size: 1942 ===
2025-06-03 18:59:47 [warning  ] loaded pipeline does not match hash
=== Generating recommendations for 1942 users ===
=== recommendations: 1942 ===
=== fold 4 ===
=== Train size: 6843397, Test size: 1942 ===
2025-06-03 19:00:39 [warning  ] loaded pipeline does not match hash
=== Generating recommendations for 1942 users ===
=== recommendations: 1942 ===

=== min_sim = 0.1 ===
=== fold 0 ===
=== Train size: 6838777, Test size: 1943 ===
2025-

/home/toporek3112/miniconda3/envs/ml-3.13/lib/python3.13/site-packages/lenskit/pipeline/_impl.py:233: PipelineWarning: loaded pipeline config does not match hash
  builder = PipelineBuilder.from_config(config)


=== Generating recommendations for 1943 users ===
=== recommendations: 1943 ===
=== fold 1 ===
=== Train size: 6842660, Test size: 1943 ===
2025-06-03 19:01:57 [warning  ] loaded pipeline does not match hash
=== Generating recommendations for 1943 users ===
=== recommendations: 1943 ===
=== fold 2 ===
=== Train size: 6844516, Test size: 1943 ===
2025-06-03 19:02:26 [warning  ] loaded pipeline does not match hash
=== Generating recommendations for 1943 users ===
=== recommendations: 1943 ===
=== fold 3 ===
=== Train size: 6843624, Test size: 1942 ===
2025-06-03 19:02:55 [warning  ] loaded pipeline does not match hash
=== Generating recommendations for 1942 users ===
=== recommendations: 1942 ===
=== fold 4 ===
=== Train size: 6843397, Test size: 1942 ===
2025-06-03 19:03:24 [warning  ] loaded pipeline does not match hash
=== Generating recommendations for 1942 users ===
=== recommendations: 1942 ===

=== min_sim = 0.5 ===
=== fold 0 ===
=== Train size: 6838777, Test size: 1943 ===
2025-

/home/toporek3112/miniconda3/envs/ml-3.13/lib/python3.13/site-packages/lenskit/pipeline/_impl.py:233: PipelineWarning: loaded pipeline config does not match hash
  builder = PipelineBuilder.from_config(config)


=== Generating recommendations for 1943 users ===
=== recommendations: 1943 ===
=== fold 1 ===
=== Train size: 6842660, Test size: 1943 ===
2025-06-03 19:04:16 [warning  ] loaded pipeline does not match hash
=== Generating recommendations for 1943 users ===
=== recommendations: 1943 ===
=== fold 2 ===
=== Train size: 6844516, Test size: 1943 ===
2025-06-03 19:04:37 [warning  ] loaded pipeline does not match hash
=== Generating recommendations for 1943 users ===
=== recommendations: 1943 ===
=== fold 3 ===
=== Train size: 6843624, Test size: 1942 ===
2025-06-03 19:05:02 [warning  ] loaded pipeline does not match hash
=== Generating recommendations for 1942 users ===
=== recommendations: 1942 ===
=== fold 4 ===
=== Train size: 6843397, Test size: 1942 ===
2025-06-03 19:05:27 [warning  ] loaded pipeline does not match hash
=== Generating recommendations for 1942 users ===
=== recommendations: 1942 ===


In [15]:
for k, res in results.items():
  print(f"\n=== min_sim = {k} ===")
  print(measure_performance(res[0]['test'], res[0]['rec']))
  print("Average recommendations per user:", res[0]['avg_rec'])
  print('---')


=== min_sim = 1e-06 ===
   Precision    Recall      NDCG       Hit       MSE      RMSE
0   0.263667  0.018468  0.049006  0.842273  0.321095  0.566653
Average recommendations per user: 50.01544799176107
---

=== min_sim = 0.005 ===
   Precision    Recall      NDCG       Hit       MSE      RMSE
0   0.263667  0.018468  0.049006  0.842273  0.321095  0.566653
Average recommendations per user: 50.01544799176107
---

=== min_sim = 0.01 ===
   Precision    Recall      NDCG       Hit       MSE      RMSE
0   0.263667  0.018468  0.049006  0.842273  0.321095  0.566653
Average recommendations per user: 50.01544799176107
---

=== min_sim = 0.05 ===
   Precision    Recall      NDCG       Hit       MSE      RMSE
0   0.262174  0.018309  0.048548  0.839288  0.319925  0.565619
Average recommendations per user: 50.01544799176107
---

=== min_sim = 0.07 ===
   Precision    Recall      NDCG       Hit       MSE      RMSE
0   0.226717  0.015279  0.039049  0.763101  0.304535  0.551847
Average recommendations 

## Final Model

In [16]:
from lenskit.knn import ItemKNNScorer
from lenskit.pipeline import RecPipelineBuilder
from lenskit.basic import UnratedTrainingItemsCandidateSelector
from lenskit import recommend

# Build and train final model
builder = RecPipelineBuilder()
scorer = ItemKNNScorer(min_sim=0.1) 
builder.scorer(scorer)
builder.candidate_selector(UnratedTrainingItemsCandidateSelector())
builder.ranker(n=10) 
pipe = builder.build("Final ItemKNN Pipeline")
pipe.train(lk_dataset)

### Export

In [21]:
import pickle
export_name = 'item-based-collaborative-filtering.pkl'

with open(export_name, 'wb') as f:
    pickle.dump(pipe, f)

### Import and recommend

In [22]:
with open(export_name, 'rb') as f:
    imported_model = pickle.load(f)

In [23]:
from lenskit.data import RecQuery, ItemList

# Suppose interactions_df is a pandas DataFrame of 10 rated movies for a user.
# Ensure it has columns 'user_id', 'item_id', 'rating', etc. (rename if necessary).
user_hist_df = movies_ratings_filtered[10:20].copy()
user_hist_df.rename(columns={'userId': 'user_id', 'movieId': 'item_id'}, inplace=True)

# Create an ItemList for the user's history (dropping the user column):
hist_items = ItemList.from_df(user_hist_df, keep_user=False)

query = RecQuery(user_id=9999, user_items=hist_items)

# Now get top-N recommendations for this query:
rec = recommend(imported_model, query, n=10)
rec_df = rec.to_df()
rec_df.merge(movies, left_on='item_id', right_on='movieId', how='left')[['movieId', 'title', 'genres']]

,movieId,title,genres
0,2905,Sanjuro (Tsubaki Sanjûrô) (1962),Action|Adventure|Drama
1,3030,Yojimbo (1961),Action|Adventure
2,1260,M (1931),Crime|Film-Noir|Thriller
3,5291,Rashomon (Rashômon) (1950),Crime|Drama|Mystery
4,3677,Baraka (1992),Documentary
5,1217,Ran (1985),Drama|War
6,926,All About Eve (1950),Drama
7,1209,Once Upon a Time in the West (C'era una volta ...,Action|Drama|Western
8,5971,My Neighbor Totoro (Tonari no Totoro) (1988),Animation|Children|Drama|Fantasy
9,1281,"Great Dictator, The (1940)",Comedy|Drama|War


In [24]:
user_hist_df.sort_values('rating', ascending=False)


,user_id,item_id,rating,timestamp,title,genres
274,3,745,5.0,1439474467,Wallace & Gromit: A Close Shave (1995),Animation|Children|Comedy
270,3,541,5.0,1439474625,Blade Runner (1982),Action|Sci-Fi|Thriller
273,3,741,5.0,1484753808,Ghost in the Shell (Kôkaku kidôtai) (1995),Animation|Sci-Fi
265,3,318,4.0,1439472424,"Shawshank Redemption, The (1994)",Crime|Drama
266,3,356,4.0,1439472199,Forrest Gump (1994),Comedy|Drama|Romance|War
269,3,527,4.0,1439472436,Schindler's List (1993),Drama|War
272,3,593,4.0,1439472203,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller
271,3,589,4.0,1439474657,Terminator 2: Judgment Day (1991),Action|Sci-Fi
267,3,442,3.5,1439474867,Demolition Man (1993),Action|Adventure|Sci-Fi
268,3,480,2.0,1439472219,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller
